# Mecánica Pulmonar: Análisis sin Surfactante

## 1. Sin Surfactante

### Energía Total (Muy Esquemática)

La funcional de energía potencial total del sistema se expresa como:

$$\Pi_0[\chi] = \int_{\Omega_0} W(F)\,dV - \int_{\Omega_t} p_{\text{alv}}\, dV - \int_{\Omega_0} \rho_0\, \mathbf{g}\cdot \chi\, dV - \int_{\Gamma_{\text{ext},0}} \mathbf{t}_{\text{pleural}}\cdot \chi\, dA_0$$

#### Componentes de la Funcional:

| Término | Símbolo | Descripción |
|---------|---------|-------------|
| **Deformación hiperelástica** | $W(F)$ | Energía de deformación (Neo-Hooke, etc.) |
| **Presión alveolar** | $p_{\text{alv}}$ | Presión interna en los alvéolos (airway pressure) |
| **Gravedad** | $\mathbf{g}$ | Campo gravitacional |
| **Tracción pleural** | $\mathbf{t}_{\text{pleural}}$ | Tracción en la superficie externa (pleural) |

---

## 2. Configuración de Equilibrio sin Surfactante

La configuración de equilibrio $\chi_0$ se obtiene como la solución del problema variacional que minimiza $\Pi_0[\chi]$.

### Ecuación de Equilibrio en el Volumen

En el interior del tejido pulmonar $\Omega_0$, se cumple la ecuación de balance de momentum:

$$\text{Div}_X\,P(F) + \rho_0\,\mathbf{g} = \mathbf{0} \quad\text{en } \Omega_0$$

donde:
- $\text{Div}_X$ es la divergencia respecto a las coordenadas materiales $X$
- $P(F) = \frac{\partial W}{\partial F}$ es el **tensor de tensiones de Piola** (o primer tensor de esfuerzos de Piola-Kirchhoff)
- $\rho_0$ es la densidad material
- $\mathbf{g}$ es el vector aceleración gravitacional

### Condiciones de Frontera

#### En la Superficie Alveolar Interna ($\Gamma_{\text{alv},0}$)

En el borde del tejido en contacto directo con el aire alveolar, se aplica:

$$P(F)\,\mathbf{N} = -p_{\text{alv}}\, J\,F^{-T}\mathbf{N}$$

donde:
- $\mathbf{N}$ es la normal exterior en la configuración material
- $J = \det(F)$ es el Jacobiano de la deformación
- $F^{-T} = (F^{-1})^T$ es la transpuesta inversa del gradiente de deformación

**Interpretación:** Solo la presión interna alveolar actúa como tracción normal sobre el tejido. No hay contribución de energía superficial (surfactante) en esta versión simplificada.

#### Configuración de Referencia

La configuración sin surfactante ($\chi_0$) representa el **estado de equilibrio bajo cargas mecánicas** pero **sin efectos de tensión superficial**. Este es el punto de partida para posteriores análisis que incluyan la acción del surfactante pulmonar.

---

## 3. Notas Importantes

- La ausencia de surfactante implica que la única contribución energética de interfaz aire-líquido es a través de la presión alveolar
- La energía hiperelástica $W(F)$ se asume independiente de la curvatura (sin bending stiffness)
- Este modelo es válido para deformaciones moderadas donde la aproximación hiperelástica es razonable
- La gravedad y la tracción pleural proporcionan cargas externas distribuidas y concentradas respectivamente

In [1]:
from mpi4py import MPI
import numpy as np
from dolfinx import mesh, io
from dolfinx.cpp.mesh import CellType
import os
import tempfile
import shutil

# --- Método GMSH (Mantenido pero no usado en el bloque principal por simplicidad) ---

def convert_gmsh_to_xdmf(gmsh_file, xdmf_file, cell_type=CellType.triangle):
    """
    Convierte archivo .msh de GMSH a formato XDMF para dolfinx.
    """
    try:
        import meshio
    except ImportError:
        raise ImportError("meshio no está instalado. Ejecuta: pip install meshio")
    try:
        msh = meshio.read(gmsh_file)
        target_cell_type = "triangle" if cell_type == CellType.triangle else "quad"

        cell_data = [cell.data for cell in msh.cells if cell.type == target_cell_type]
        if not cell_data:
            raise ValueError(f"No se encontraron elementos '{target_cell_type}' en la malla")
        
        cell_data = np.vstack(cell_data)
        os.makedirs(os.path.dirname(xdmf_file) or '.', exist_ok=True)
        
        mesh_meshio = meshio.Mesh(
            points=msh.points[:, :2], 
            cells=[(target_cell_type, cell_data)],
            cell_data={"name_to_read": [np.ones(cell_data.shape[0])]},
            field_data=msh.field_data
        )
        mesh_meshio.write(xdmf_file)
    except Exception as e:
        print(f"Error en convert_gmsh_to_xdmf: {e}")
        raise

def create_annulus_mesh(R_in, R_out, mesh_resolution=0.1, filename=None):
    """
    Crea una malla 2D de un anillo (domino circular con agujero) usando GMSH.
    """
    try:
        import gmsh
    except ImportError:
        raise ImportError("gmsh no está instalado. Ejecuta: pip install gmsh")

    if filename is None:
        filename = "annulus_gmsh_mesh.xdmf"
    
    gmsh.initialize()
    gmsh.option.setNumber("General.Terminal", 0)
    
    temp_dir = tempfile.mkdtemp()
    msh_path = os.path.join(temp_dir, "annulus_mesh.msh")
    
    try:
        gmsh.model.add("annulus")
        
        # 1. Crear geometría del anillo
        center_point = gmsh.model.geo.addPoint(0, 0, 0, mesh_resolution)
        
        p_out = [(gmsh.model.geo.addPoint(R_out, 0, 0, mesh_resolution)),
                 (gmsh.model.geo.addPoint(0, R_out, 0, mesh_resolution)),
                 (gmsh.model.geo.addPoint(-R_out, 0, 0, mesh_resolution)),
                 (gmsh.model.geo.addPoint(0, -R_out, 0, mesh_resolution))]
        
        outer_arcs = [gmsh.model.geo.addCircleArc(p_out[i], center_point, p_out[(i+1) % 4]) for i in range(4)]
        outer_loop = gmsh.model.geo.addCurveLoop(outer_arcs)
        
        p_in = [(gmsh.model.geo.addPoint(R_in, 0, 0, mesh_resolution)),
                (gmsh.model.geo.addPoint(0, R_in, 0, mesh_resolution)),
                (gmsh.model.geo.addPoint(-R_in, 0, 0, mesh_resolution)),
                (gmsh.model.geo.addPoint(0, -R_in, 0, mesh_resolution))]
                
        inner_arcs = [gmsh.model.geo.addCircleArc(p_in[i], center_point, p_in[(i+1) % 4]) for i in range(4)]
        inner_loop = gmsh.model.geo.addCurveLoop(inner_arcs)
        
        surface = gmsh.model.geo.addPlaneSurface([outer_loop, inner_loop])
        
        # 2. Sincronizar y generar malla
        gmsh.model.geo.synchronize()
        
        # 3. Asignar nombres físicos para las fronteras (para condiciones de contorno)
        gmsh.model.addPhysicalGroup(1, outer_arcs, 1, name="Gamma_out")
        gmsh.model.addPhysicalGroup(1, inner_arcs, 2, name="Gamma_in")
        gmsh.model.addPhysicalGroup(2, [surface], 3, name="Omega")
        
        gmsh.model.mesh.generate(2)
        
        gmsh.write(msh_path)
        convert_gmsh_to_xdmf(msh_path, filename, cell_type=CellType.triangle)
        
        print(f"✓ Malla de anillo (GMSH) creada: R_in={R_in}, R_out={R_out}")
        print(f"✓ Archivo guardado: {filename}")
        
        return filename
        
    except Exception as e:
        print(f"❌ Error creando malla con GMSH: {e}")
        raise
    finally:
        gmsh.finalize()
        if os.path.exists(temp_dir):
            shutil.rmtree(temp_dir)

# --- Método 2: Mallas simples (Recomendado por su robustez) ---

def create_simple_annulus_mesh(R_in=0.1, R_out=1.0, n_circumferential=64, n_radial=16):
    """
    Crea una malla simple de anillo (cuadriláteros mapeados) usando solo dolfinx.
    """
    comm = MPI.COMM_WORLD
    
    # Crear malla rectangular [theta_min, theta_max] x [r_min, r_max]
    n_cells = [n_circumferential, n_radial]
    domain = mesh.create_rectangle(comm, 
                                   [np.array([0, R_in]), np.array([2*np.pi, R_out])], 
                                   n_cells, 
                                   mesh.CellType.quadrilateral)
    
    # Transformar a coordenadas polares a cartesianas, para rotar y asi anillo
    points = domain.geometry.x.copy()
    r = points[:, 1]       # Coordenada radial (y en el rectángulo base)
    theta = points[:, 0]   # Coordenada angular (x en el rectángulo base)
    
    points[:, 0] = r * np.cos(theta)
    points[:, 1] = r * np.sin(theta)
    
    domain.geometry.x[:, :] = points[:, :]
    
    return domain

def create_simple_slab_mesh(W=2.0, H=1.0, n_x=64, n_y=16):
    """
    Crea una malla simple de placa o slab (rectángulo) usando solo dolfinx.
    """
    comm = MPI.COMM_WORLD
    # Crear malla rectangular [x_min, y_min] x [x_max, y_max]
    domain = mesh.create_rectangle(comm, 
                                   [np.array([0, 0]), np.array([W, H])], 
                                   [n_x, n_y], 
                                   mesh.CellType.quadrilateral)
    
    return domain

# --- Carga y Visualización ---

def load_and_visualize_mesh(filename):
    """Carga y visualiza la malla generada"""
    comm = MPI.COMM_WORLD
    
    try:
        with io.XDMFFile(comm, filename, "r") as xdmf:
            domain = xdmf.read_mesh() 
        
        print(f"\n=== INFORMACIÓN DE LA MALLA CARGADA ===")
        print(f"Dimensiones de los puntos: {domain.geometry.x.shape}")
        print(f"Tipo de celda: {domain.topology.cell_name()}")
        print(f"Número de elementos globales: {domain.topology.index_map(domain.topology.dim).size_global}")
        
        coords = domain.geometry.x
        
        if np.isclose(coords[:,0].min(), 0.0) and np.isclose(coords[:,1].min(), 0.0):
            print(f"Ancho (max X): {coords[:,0].max():.3f}")
            print(f"Alto (max Y): {coords[:,1].max():.3f}")
        else:
            # Asumimos Anillo o geometría centrada
            radii = np.sqrt(coords[:,0]**2 + coords[:,1]**2)
            print(f"Radio mínimo: {np.min(radii):.3f}")
            print(f"Radio máximo: {np.max(radii):.3f}")
        
        return domain
    except Exception as e:
        print(f"❌ Error cargando malla: {e}")
        return None

GEOMETRY_TYPE = "ANNULUS"  # Para el anillo

# Parámetros del ANILLO
R_in = 0.1     
R_out = 1.0    

# Parámetros del SLAB
SLAB_W = 2.0   # Ancho
SLAB_H = 1.0   # Alto

if __name__ == "__main__":
    print("="*50)
    print("=== GENERADOR MALLA 2D (DOLFINX) ===")
    print(f"GEOMETRÍA SELECCIONADA: {GEOMETRY_TYPE}")
    print("="*50)
    
    domain_to_save = None
    output_filename = f"simple_{GEOMETRY_TYPE.lower()}_mesh.xdmf"

    try:
        print(f"\n--- Intentando Método Simple para {GEOMETRY_TYPE} ---")
        
        if GEOMETRY_TYPE == "ANNULUS":
            print(f"Parámetros: R_in={R_in}, R_out={R_out}")
            domain_to_save = create_simple_annulus_mesh(R_in, R_out)
        elif GEOMETRY_TYPE == "SLAB":
            print(f"Parámetros: Ancho={SLAB_W}, Alto={SLAB_H}")
            domain_to_save = create_simple_slab_mesh(SLAB_W, SLAB_H)
        else:
            raise ValueError("GEOMETRY_TYPE no válido. Debe ser 'ANNULUS' o 'SLAB'.")
        
        # Guardar malla
        with io.XDMFFile(MPI.COMM_WORLD, output_filename, "w") as xdmf:
            xdmf.write_mesh(domain_to_save)
        
        print(f"✓ Malla simple guardada en: {output_filename}")
        
        # Visualizar malla
        load_and_visualize_mesh(output_filename)
        
    except Exception as e:
        print(f"❌ Error durante la generación: {e}")
        print("\n--- Intentando Método GMSH como alternativa (solo para ANNULUS) ---")
        
        try:
            if GEOMETRY_TYPE == "ANNULUS":
                mesh_filename_gmsh = create_annulus_mesh(R_in, R_out, mesh_resolution=0.05)
                load_and_visualize_mesh(mesh_filename_gmsh)
            else:
                print("El método GMSH solo está implementado para la geometría ANNULUS.")
        except Exception as e2:
            print(f"❌ Error con GMSH: {e2}")
            print("💡 No se pudo generar la malla. Asegúrate de tener 'gmsh' y 'meshio' instalados.")
            
    
    print("\n" + "="*50)
    print("PROCESO DE GENERACIÓN DE MALLA COMPLETADO")
    print("="*50)

ModuleNotFoundError: No module named 'dolfinx'

In [ ]:
from mpi4py import MPI
from petsc4py import PETSc
import numpy as np
from basix.ufl import element, mixed_element
from dolfinx import fem, io, mesh
from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver
from ufl import (Identity, TestFunctions, TrialFunctions, split, 
                 grad, det, tr, inner, derivative, dx, variable, inv, ln, TrialFunction)
import ufl
import os
import tempfile
import pickle
from basix.cell import CellType

# Cargar Malla! (Anillo)
MESH_FILENAME = "simple_annulus_mesh.xdmf"

try:
    with io.XDMFFile(MPI.COMM_WORLD, MESH_FILENAME, "r") as xdmf:
        domain = xdmf.read_mesh()
    print(f"✓ Malla cargada: {MESH_FILENAME}")
    
    R_in = 0.1
    R_out = 1.0

except Exception as e:
    print(f"❌ Error al cargar {MESH_FILENAME}: {e}. Generando malla de SLAB de respaldo.")
    domain = mesh.create_unit_square(MPI.COMM_WORLD, 32, 32, mesh.CellType.triangle)
    R_in = 0.0 # Parámetros ajustados para el SLAB
    R_out = 1.0


# ELEMENTOS ROBUSTOS
k = 2
V_el = element("Lagrange", domain.basix_cell(), k, shape=(domain.geometry.dim,))
Q_el = element("Lagrange", domain.basix_cell(), k-1)
V_mixed = mixed_element([V_el, Q_el])
V = fem.functionspace(domain, V_mixed)

# PARÁMETROS 
mu = fem.Constant(domain, 1.0)
lmbda = fem.Constant(domain, 100.0)

# Función para la solución
u_p_ = fem.Function(V)
u, p = split(u_p_)

# Funciones test
v, q = TestFunctions(V)

# --- Neo-Hooke mixto (u, p) ---

d = domain.geometry.dim
I = Identity(d)

# Desplazamiento u y presión p ya definidos como Function en su espacio
# u_p_ = Function(W) con split(u_p_) -> (u, p)

F = ufl.variable(I + grad(u))
J = ufl.det(F)
C = F.T * F
Ic = ufl.tr(C)

# Parte isocórica (deviatoric) tipo Neo-Hooke modificado
psi_iso = (mu / 2) * (Ic - 2 - 2 * ufl.ln(J))

# Parte volumétrica en formulación mixta
psi_vol = -p * ufl.ln(J) + (1.0 / (2.0 * lmbda)) * p**2

psi = psi_iso + psi_vol

# Piola de 1ª especie
P = ufl.diff(psi, F)

# Forma débil: equilibrio + ecuación de estado (ln J - p/λ = 0)
G = (
    inner(P, grad(v)) * dx
    + (ufl.ln(J) - p / lmbda) * q * dx
)


# Jacobiano para Newton
du = TrialFunction(V)          # V: espacio de desplazamientos dentro de W
J_form = derivative(G, u_p_, du)


# CONDICIONES DE BORDE, Readaptar considerando cuerpos complejos posiblemente, por ahora anillo

def inner_boundary(x):
    r = np.sqrt(x[0]**2 + x[1]**2)
    # R_in es 0.1 en la malla de anillo.
    return np.isclose(r, R_in)

def outer_boundary(x):
    r = np.sqrt(x[0]**2 + x[1]**2)
    # R_out es 1.0 en la malla de anillo.
    return np.isclose(r, R_out)

inner_facets = mesh.locate_entities_boundary(domain, domain.topology.dim-1, inner_boundary)
outer_facets = mesh.locate_entities_boundary(domain, domain.topology.dim-1, outer_boundary)

# Fija u_x = 0 y u_y = 0 en la cara exterior!
V0_outer, _ = V.sub(0).collapse()
outer_dofs = fem.locate_dofs_topological((V.sub(0), V0_outer), domain.topology.dim-1, outer_facets)

u_outer = fem.Function(V0_outer)
u_outer.x.array[:] = 0.0
bc_outer = fem.dirichletbc(u_outer, outer_dofs, V.sub(0))

# Presión alveolar aplicada (Carga de Neumann)
p_alv_const = fem.Constant(domain, 0.0)

# Etiquetar la frontera interna
fdim = domain.topology.dim - 1
marked_facets = np.hstack([inner_facets])
marked_values = np.hstack([np.full_like(inner_facets, 1)])
sorted_facets = np.argsort(marked_facets)
facet_tag = mesh.meshtags(domain, fdim, marked_facets[sorted_facets], marked_values[sorted_facets])
ds = ufl.Measure("ds", domain=domain, subdomain_data=facet_tag)

# Agregar término de presión (Neumann) a la forma débil
n = ufl.FacetNormal(domain)
G += inner(-p_alv_const * J * inv(F).T * n, v) * ds(1)

bcs = [bc_outer]
print("✓ BCs corregidas: Borde externo fijo (Dirichlet). Presión interna (Neumann) aplicada en borde interno.")

# SOLVER CONFIGURADO
problem = NonlinearProblem(G, u_p_, bcs=bcs, J=J_form)
solver = NewtonSolver(MPI.COMM_WORLD, problem)
solver.atol = 1e-8
solver.rtol = 1e-8
solver.max_it = 50
solver.convergence_criterion = "incremental"

ksp = solver.krylov_solver
opts = PETSc.Options()
option_prefix = ksp.getOptionsPrefix()
opts[f"{option_prefix}ksp_type"] = "gmres"
opts[f"{option_prefix}pc_type"] = "lu"
opts[f"{option_prefix}pc_factor_mat_solver_type"] = "mumps"
ksp.setFromOptions()

# CARGA ADAPTATIVA
def adaptive_load_increments(initial_disp, target_disp, max_steps=50):
    increments = []
    current = initial_disp
    
    while current < target_disp:
        increments.append(current)
        if current < target_disp * 0.1:
            step = target_disp / 30
        elif current < target_disp * 0.5:
            step = target_disp / 20
        else:
            step = target_disp / 15
            
        current += step
        if current > target_disp:
            current = target_disp
    
    increments.append(target_disp)
    return np.unique(increments)

initial_pressure = 0.01
target_pressure = 1.0
max_steps = 100

load_values = adaptive_load_increments(initial_pressure, target_pressure, max_steps)
num_steps = len(load_values) - 1

print(f"Estrategia de carga adaptativa:")
print(f"   - Presión objetivo: {target_pressure:.3f}")
print(f"   - Número de pasos: {num_steps}")

# Preparar espacios de visualización P1
V_vis = fem.functionspace(domain, ("Lagrange", 1, (domain.geometry.dim,)))
Q_vis = fem.functionspace(domain, ("Lagrange", 1))

print("\n=== Iniciando simulación directa ===")

convergence_history = []
all_converged = True

for i, pressure in enumerate(load_values[1:]):
    step_size = pressure - load_values[i]
    print(f"Paso {i+1}/{num_steps}, Presión = {pressure:.4f} (Δ = {step_size:.4f})")
    
    p_alv_const.value = pressure
    
    try:
        num_its, converged = solver.solve(u_p_)
        
        if converged:
            print(f"   ✓ Converged in {num_its} iterations")
            convergence_history.append(num_its)
            
            u_sol = u_p_.sub(0).collapse()
            u_mag = fem.Function(Q_vis)
            u_mag.interpolate(fem.Expression(ufl.sqrt(inner(u_sol, u_sol)), Q_vis.element.interpolation_points()))
            max_deformation = np.max(u_mag.x.array)

            print(f"   → Deformación máxima (Magnitud): {max_deformation:.4f}")
            
        else:
            print(f"   ✗ FAILED to converge after {num_its} iterations")
            all_converged = False
            break
            
    except Exception as e:
        print(f"   ✗ ERROR: {e}")
        all_converged = False
        break

print("\n=== Guardando resultados ===")

if all_converged:
    
    print("--- Guardando datos para problema inverso (Alta fidelidad) ---")
    
    u_sol = u_p_.sub(0).collapse()
    p_sol = u_p_.sub(1).collapse()
    u_sol.name = "Displacement"
    p_sol.name = "Pressure"

    # --- 2. DATOS PARA VISUALIZACIÓN EN XDMF (P1) ---
    print("\n--- Guardando resultados en XDMF para visualización (P1) ---")
    
    # Interpolar a P1 para XDMF
    u_vis = fem.Function(V_vis)
    u_vis.name = "Displacement"
    u_vis.interpolate(u_sol)
    
    p_vis = fem.Function(Q_vis)
    p_vis.name = "Pressure"
    p_vis.interpolate(p_sol)
    
    # Guardar en XDMF (malla original)
    xdmf_file = io.XDMFFile(domain.comm, os.path.join("simulation_results.xdmf"), "w")
    xdmf_file.write_mesh(domain)
    xdmf_file.write_function(u_vis, 0.0)
    xdmf_file.write_function(p_vis, 0.0)
    xdmf_file.close()
    print("✓ XDMF guardado: simulation_results.xdmf")
    
    print("\n--- Creando malla deformada para visualización ---")
        
    # 1. Calcular nuevas coordenadas
    original_coords = domain.geometry.x
    displacement_at_vertices = u_vis.x.array.reshape((-1, domain.geometry.dim))
    
    displacement_3d = np.zeros_like(original_coords)
    displacement_3d[:, :domain.geometry.dim] = displacement_at_vertices
    new_coords = original_coords + displacement_3d
    
    # 2. CORRECCIÓN DE CONECTIVIDAD (Sin usar basix explícitamente)
    # Obtenemos la topología de la malla actual
    tdim = domain.topology.dim
    domain.topology.create_connectivity(tdim, 0) # Generar mapa Celdas -> Vértices
    connectivity = domain.topology.connectivity(tdim, 0)
    
    # Contamos cuántos vértices tiene la primera celda para saber el tamaño (ej. 3 para triángulos)
    num_verts_per_cell = connectivity.links(0).size
    
    # Reconstruimos el array de celdas
    cells_array = connectivity.array.reshape((-1, num_verts_per_cell))
    cells_array = np.asarray(cells_array, dtype=np.int64)
    
    # 3. Crear la malla deformada
    deformed_mesh = mesh.create_mesh(domain.comm, cells_array, new_coords[:, :domain.geometry.dim], domain.ufl_domain())
    
    # Crear funciones en malla deformada para guardar los datos
    V_vis_deformed = fem.functionspace(deformed_mesh, ("Lagrange", 1, (deformed_mesh.geometry.dim,)))
    Q_vis_deformed = fem.functionspace(deformed_mesh, ("Lagrange", 1))
    
    u_vis_def = fem.Function(V_vis_deformed)
    u_vis_def.name = "Displacement"
    u_vis_def.x.array[:] = u_vis.x.array
    
    p_vis_def = fem.Function(Q_vis_deformed)
    p_vis_def.name = "Pressure"
    p_vis_def.x.array[:] = p_vis.x.array
    
    # Guardar malla deformada en XDMF
    xdmf_deformed = io.XDMFFile(deformed_mesh.comm, os.path.join("deformed_results.xdmf"), "w")
    xdmf_deformed.write_mesh(deformed_mesh)
    xdmf_deformed.write_function(u_vis_def, 0.0)
    xdmf_deformed.write_function(p_vis_def, 0.0)
    xdmf_deformed.close()
    print("✓ XDMF deformado guardado: deformed_results.xdmf")

else:
    print(f"✗ Simulación no convergente. No se guardaron resultados en: deformed_results.xdmf")

Results will be saved in: /tmp/fem_results_6jlbt10y
✓ BCs corregidas: Cara inferior fija, Cara superior desliza en X.
Estrategia de carga adaptativa:
   - Desplazamiento objetivo: 0.100
   - Número de pasos: 17

=== Iniciando simulación directa ===
Paso 1/17, Desplazamiento = 0.0133 (Δ = 0.0033)
   ✓ Converged in 4 iterations
   → Deformación máxima Y: 0.0133
   → Deformación máxima X (abombamiento): 0.0080
Paso 2/17, Desplazamiento = 0.0183 (Δ = 0.0050)
   ✓ Converged in 4 iterations
   → Deformación máxima Y: 0.0183
   → Deformación máxima X (abombamiento): 0.0111
Paso 3/17, Desplazamiento = 0.0233 (Δ = 0.0050)
   ✓ Converged in 4 iterations
   → Deformación máxima Y: 0.0233
   → Deformación máxima X (abombamiento): 0.0142
Paso 4/17, Desplazamiento = 0.0283 (Δ = 0.0050)
   ✓ Converged in 4 iterations
   → Deformación máxima Y: 0.0283
   → Deformación máxima X (abombamiento): 0.0173
Paso 5/17, Desplazamiento = 0.0333 (Δ = 0.0050)
   ✓ Converged in 4 iterations
   → Deformación máxima 

# Mecánica Pulmonar: Análisis con Surfactante

## 2. Con Surfactante

### Funcional de Energía con Contribución Superficial

Se añade a la funcional anterior una contribución de energía de superficie en la interfaz fluido–sólido (pared alveolar) debida al surfactante pulmonar:

$$\Pi_{\text{surf}}[\chi] = \Pi_0[\chi] + \int_{\Gamma_{\text{alv},0}} \gamma(J_s)\, dA_0$$

#### Nuevos Parámetros:

| Símbolo | Descripción |
|---------|-------------|
| $\gamma(J_s)$ | Tensión superficial efectiva (dependiente del stretch superficial) |
| $J_s$ | Determinante tangencial del gradiente de deformación en $\Gamma_{\text{alv},0}$ (mide el estiramiento de la superficie) |
| $A$ | Área actual de la interfaz |

**Característica clave:** La tensión superficial es **variable**, dependiendo del área (o del stretch) de la interfaz.

---

## 3. Ecuaciones de Equilibrio con Surfactante

### Ecuación de Equilibrio en el Volumen

La ecuación en el volumen **permanece igual**:

$$\text{Div}_X\,P(F) + \rho_0\,\mathbf{g} = \mathbf{0} \quad\text{en } \Omega_0$$

### Condición de Borde Modificada en la Interfaz Alveolar

La variación de $\Pi_{\text{surf}}$ produce una condición de borde interna **modificada**:

$$P(F)\,\mathbf{N} + \mathbf{t}_{\text{surf}}(F) = -p_{\text{alv}}\, J\,F^{-T}\mathbf{N} \quad\text{en } \Gamma_{\text{alv},0}$$

donde:
- $\mathbf{t}_{\text{surf}}(F)$ es la **tracción adicional debida al surfactante**
- Esta tracción surge de la variación de la energía superficial

---

## 4. Aproximaciones Especiales

### 4.1 Tensión Superficial Constante (Ley de Laplace Clásica)

Si $\gamma$ es **constante** (independiente del área), la tracción superficial toma la forma típica de la **ley de Young-Laplace**:

$$\mathbf{t}_{\text{surf}} \sim 2\gamma\,H\,\mathbf{n}$$

donde:
- $\gamma$ es la tensión superficial constante
- $H$ es la **curvatura media** de la interfaz
- $\mathbf{n}$ es la normal en la configuración actual
- El factor $2H$ representa la suma de curvaturas principales

**Interpretación física:** La presencia de una interfaz curva con tensión superficial produce una tracción normal que depende de la geometría local.

### 4.2 Tensión Superficial Dependiente del Área (Modelo Realista)

Cuando $\gamma$ depende del estiramiento superficial $J_s$:

$$\gamma = \gamma(J_s)$$

La tracción superficial contiene términos adicionales proporcionales a $\frac{d\gamma}{dJ_s}$. Conceptualmente:

$$\mathbf{t}_{\text{surf}} = 2\gamma(J_s)\,H\,\mathbf{n} + \text{términos adicionales con } \frac{d\gamma}{dJ_s}$$

**Característica importante:** El surfactante pulmonar real se comporta como un **resorte que se "ablanda"** cuando la interfaz se estira (es decir, $\frac{d\gamma}{dJ_s} < 0$). Esto reduce la resistencia mecánica del pulmón durante la inspiración.

---

## 5. Solución: Configuración con Surfactante

La configuración de equilibrio con surfactante $\chi_{\text{surf}}$ es la **solución del mismo problema de equilibrio** pero con la **condición de borde modificada**:

$$\begin{cases}
\text{Div}_X\,P(F) + \rho_0\,\mathbf{g} = \mathbf{0} & \text{en } \Omega_0 \\
P(F)\,\mathbf{N} + \mathbf{t}_{\text{surf}}(F) = -p_{\text{alv}}\, J\,F^{-T}\mathbf{N} & \text{en } \Gamma_{\text{alv},0} \\
\cdots & \text{(otras condiciones de borde)}
\end{cases}$$

### Diferencia Crucial

Para una **misma presión alveolar** $p_{\text{alv}}$, las configuraciones:

$$\chi_{\text{surf}} \quad\text{y}\quad \chi_0$$

**no coinciden en general**. La diferencia es atribuible al efecto mecánico del surfactante:
- El surfactante modifica la **deformación local** del tejido
- Cambia la **rigidez efectiva** de las paredes alveolares
- Afecta la **distribución de esfuerzos** en toda la estructura

---

## 6. Implicaciones Fisiológicas

| Aspecto | Sin Surfactante | Con Surfactante |
|--------|-----------------|-----------------|
| **Tensión superficial** | No existe | Variable con el área |
| **Tracción en frontera** | Solo presión interna | Presión + contribución superficial |
| **Rigidez efectiva** | Mayor (sin "ablandamiento") | Menor (especialmente al estirar) |
| **Compliance pulmonar** | Reducida | Aumentada |
| **Energía requerida** | Mayor | Menor (efecto protector) |
| **Inestabilidad alveolar** | Posible | Prevenida |

---

## 7. Notas Adicionales

- El surfactante pulmonar es una mezcla compleja de lípidos y proteínas que en modelos continuum se simplifica como una densidad de energía superficial
- La dependencia $\gamma(J_s)$ captura el comportamiento de "phase transition" del surfactante durante ciclos de compresión-expansión
- La comparación entre $\chi_0$ y $\chi_{\text{surf}}$ permite cuantificar la **contribución mecánica del surfactante** al funcionamiento pulmonar normal

Simulacion Multiescala